# Intro/Note for the Grader
I have been having an issue with Jupyter Notebooks where vpython outputs won't show up properly, sometimes they don't show up at all and sometimes they don't show up at the right cell regardless of where I do `scene=canvas()` or `g1=graph()`

Because of this issue I was developing in a single python file running in vscode, and just pasted the sections into this notebook at the end of the process to make it more of a report. But I really can't run things in here. Also I talked to Dr. Bocan about this and she didn't know what was going on.

But so if the graph's show up properly for you then that's wonderful but if not for me they were typically showing up infront of the **Imports** sections cell.

I also decided to put all the lines into a single graph for each setup for this reason, because things would become unyieldy otherwise. For the graphs, each line has 3 y values for it, the lowest means undefined, middle means 0, and highest means 1. Each multiple of 3 will be the 0 line for that net. So if there are 3 nets then y=6 would be the 0 line for that net (and if the line is on y=5 that means undefined, y=7 means 1)

# Imports

In [1]:
from vpython import *
from queue import PriorityQueue
GATE_DELAY=10

# Custom PriorityQueue I was using because imported PriorityQueue has issues if you try to give it two events with same time
# class PriorityQueue:
#     def __init__(self):
#         self.items=[]
#     def put(self, item):
#         for i in range(len(self.items)):
#             if item < self.items[i]:
#                 self.items.insert(i, item)
#                 return
#         self.items.append(item)
#     def get(self):
#         return self.items.pop(0)
#     def empty(self):
#         return len(self.items)==0

<IPython.core.display.Javascript object>

# Class Definitions

In [2]:
class Gate:
    def __init__(self, func, inputs, output, delay=GATE_DELAY):
        self.func = func
        self.inputs = inputs
        for net in self.inputs:
            net.receivers.append(self)
        self.output = output
        self.delay = delay
    def drive(self, t):
        
        outputValue = self.func(self.inputs)
        # if t+self.delay==21:
        #     print(outputValue)
        #     print(self.output.value)
        # outputValue = None
        # for net in self.inputs:
        #     if net.value=='X':
        #         # print('X')
        #         outputValue='X'
                
        # # print(self.func(self.inputs))
        # if outputValue == None:
        #     outputValue=self.func(self.inputs)
        if self.output.value != outputValue:
            return Event(self.output.id, t+self.delay, outputValue)

def nand(inputs):
    hasX = False
    for net in inputs:
        if net.value==0:
            return 1
        elif net.value=='X':
            hasX = True
    
    return 'X' if hasX else 0

def nor(inputs):
    hasX = False
    for net in inputs:
        if net.value==1:
            return 0
        elif net.value=='X':
            hasX = True
    
    return 'X' if hasX else 1
        
    
class Event:
    def __init__(self, netID, time, value):
        self.netID = netID
        self.time = time
        self.value = value
    def __lt__(self, other):
        return self.time < other.time
    def __str__(self):
        return "Net ID: " + str(self.netID) + " | Time: " + str(self.time) + " | Value: " + str(self.value)
    
class Net:
    nets = []
    def __init__(self, receivers=None, initialValue='X'):
        self.id = len(Net.nets)
        if receivers == None:
            self.receivers = []
        else:
            self.receivers = receivers
        #self.receivers = receivers[0:-1] ## ALSO WORKS
        self.value=initialValue
        Net.nets.append(self)
    def update(self, value, t):
        self.value = value
        events = []
        for receiver in self.receivers:
            event = receiver.drive(t)
            if event:
                events.append(event)
        return events
    def graphValue(self):
        return -1+self.id*3 if self.value == 'X' else self.value+self.id*3

# Simulations

## NAND Gate

In [3]:
# Clear static variable cause jupyter notebook issues (wouldn't need in normal program)
Net.nets = []

queue = PriorityQueue()

gate = Gate(nand, [Net(), Net()], Net())

inputEvents = [Event(0, 0, 0), Event(1, 0, 0), Event(0, 0, 1), Event(1, 0, 1), Event(0, 0, 0)]
for i, event in enumerate(inputEvents):
    event.time += GATE_DELAY*3*i
[queue.put(i) for i in inputEvents]
t=0
g1 = graph(title='NAND', xtitle='time (picoseconds)', ytitle='X/0/1')
curves = [gcurve(color=vec(0, 1.0/float(i+1), 1.0/float(len(Net.nets)-i))) for i in range(len(Net.nets))]
curves[0].plot(-1,0)

runs=0

while not queue.empty():
    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    event = queue.get()
    t = event.time

    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    [queue.put(ev) for ev in Net.nets[event.netID].update(event.value, t)]
    runs+=1
    rate(20)
    
for i in range(len(curves)):
    curves[i].plot(pos=(t, Net.nets[i].graphValue()))
for i in range(len(curves)):
    curves[i].plot(pos=(t+GATE_DELAY*2, Net.nets[i].graphValue()))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done!


We can see that when either input is 0 the output is 1, then when both inputs are 1 the output is 0.

## OR Gate from 3 NAND Gates (DeMorgan's Law)
\~(AA) = \~A  
\~(BB) = \~B  
\~((\~A)(\~B)) = A+B

In [ ]:
# Clear static variable Net.nets cause jupyter notebook issues (wouldn't need in normal program)
Net.nets = []

queue = PriorityQueue()

A = Net()
B = Net()
notA = Net()
notB = Net()
AorB = Net()
notAGate = Gate(nand, [A, A], notA)
notBGate = Gate(nand, [B, B], notB)
AorBGate = Gate(nand, [notA, notB], AorB)

inputEvents = [Event(0, 0, 0), Event(1, 0, 0), Event(0, 0, 1), Event(1, 0, 1), Event(0, 0, 0)]
for i, event in enumerate(inputEvents):
    event.time += GATE_DELAY*4*i
[queue.put(i) for i in inputEvents]
t=0
g1 = graph(title='OR', xtitle='time (picoseconds)', ytitle='X/0/1')
curves = [gcurve(color=vec(0, 1.0/float(i+1), 1.0/float(len(Net.nets)-i))) for i in range(len(Net.nets))]
curves[0].plot(-1,0)

runs=0

while not queue.empty():
    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    event = queue.get()
    t = event.time

    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    [queue.put(ev) for ev in Net.nets[event.netID].update(event.value, t)]
    runs+=1
    rate(20)
    
for i in range(len(curves)):
    curves[i].plot(pos=(t, Net.nets[i].graphValue()))
for i in range(len(curves)):
    curves[i].plot(pos=(t+GATE_DELAY*2, Net.nets[i].graphValue()))


We can see that when either input is 1 the output is 1, then when both inputs are 0 the output is 0.

## NOR Gate

In [5]:
# Clear static variable cause jupyter notebook issues (wouldn't need in normal program)
Net.nets = []

queue = PriorityQueue()

gate = Gate(nor, [Net(), Net()], Net())

inputEvents = [Event(0, 0, 0), Event(1, 0, 0), Event(0, 0, 1), Event(1, 0, 1), Event(0, 0, 0)]
for i, event in enumerate(inputEvents):
    event.time += GATE_DELAY*3*i
[queue.put(i) for i in inputEvents]
t=0
g1 = graph(title='NOR', xtitle='time (picoseconds)', ytitle='X/0/1')
curves = [gcurve(color=vec(0, 1.0/float(i+1), 1.0/float(len(Net.nets)-i))) for i in range(len(Net.nets))]
curves[0].plot(-1,0)

runs=0

while not queue.empty():
    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    event = queue.get()
    t = event.time

    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    [queue.put(ev) for ev in Net.nets[event.netID].update(event.value, t)]
    runs+=1
    rate(20)
    
for i in range(len(curves)):
    curves[i].plot(pos=(t, Net.nets[i].graphValue()))
for i in range(len(curves)):
    curves[i].plot(pos=(t+GATE_DELAY*2, Net.nets[i].graphValue()))


Done!


We can see that when either input is 1 the output is 0, then when both inputs are 0 the output is 1.

## AND Gate from 3 NOR Gates (DeMorgan's Law)
\~(A+A) = \~A  
\~(B+B) = \~B  
\~(\~A+\~B) = AB

In [6]:
# Clear static variable Net.nets cause jupyter notebook issues (wouldn't need in normal program)
Net.nets = []

queue = PriorityQueue()

A = Net()
B = Net()
notA = Net()
notB = Net()
AandB = Net()
notAGate = Gate(nor, [A, A], notA)
notBGate = Gate(nor, [B, B], notB)
AandBGate = Gate(nor, [notA, notB], AandB)

inputEvents = [Event(0, 0, 0), Event(1, 0, 0), Event(0, 0, 1), Event(1, 0, 1), Event(0, 0, 0)]
for i, event in enumerate(inputEvents):
    event.time += GATE_DELAY*4*i
[queue.put(i) for i in inputEvents]
t=0
g1 = graph(title='AND', xtitle='time (picoseconds)', ytitle='X/0/1')
curves = [gcurve(color=vec(0, 1.0/float(i+1), 1.0/float(len(Net.nets)-i))) for i in range(len(Net.nets))]
curves[0].plot(-1,0)

runs=0

while not queue.empty():
    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    event = queue.get()
    t = event.time

    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    [queue.put(ev) for ev in Net.nets[event.netID].update(event.value, t)]
    runs+=1
    rate(20)
    
for i in range(len(curves)):
    curves[i].plot(pos=(t, Net.nets[i].graphValue()))
for i in range(len(curves)):
    curves[i].plot(pos=(t+GATE_DELAY*2, Net.nets[i].graphValue()))


Done!


We can see that when either input is 0 the output is 0, then when both inputs are 1 the output is 1.

## D-Latch

In [ ]:
# Clear static variable cause jupyter notebook issues (wouldn't need in normal program)
Net.nets = []

queue = PriorityQueue()
# AND from two NANDS
# gate1 = Gate(nand, [Net(), Net()], Net())
# gate2 = Gate(nand, [gate1.output, gate1.output], Net())
# initialEvents = [Event(0, 0, 0), Event(1, 1, 0), Event(0, 2, 1), Event(1, 3, 1)]

# D Latch
# D=Net(initialValue=0)
# E=Net(initialValue=0)
# notD=Net(initialValue=1)
# S=Net(initialValue=1)
# R=Net(initialValue=1)
D=Net()
E=Net()
notD=Net()
S=Net()
R=Net()
# Give Q and QBar initial values because of feedback of 'X' issues otherwise
Q=Net(initialValue=0)
QBar=Net(initialValue=1)
SGate=Gate(nand, [D, E], S)
notGate=Gate(nand, [D,D], notD)
RGate=Gate(nand, [notD,E], R)
QGate=Gate(nand, [S, QBar], Q)
QBarGate=Gate(nand, [R, Q], QBar)
inputEvents = [Event(0, 0, 0), Event(1, 0, 0), Event(0, 0, 1), Event(0, 0, 0), Event(1, 0, 1), Event(0, 0, 1), Event(0, 0, 0), Event(0, 0, 1), Event(1, 0, 0), Event(0, 0, 0)]
for i, event in enumerate(inputEvents):
    event.time += GATE_DELAY*4*i
[queue.put(i) for i in inputEvents]
t=0
g1 = graph(title='D-LATCH', xtitle='time (picoseconds)', ytitle='X/0/1')
curves = [gcurve(color=vec(0, 1.0/float(i+1), 1.0/float(len(Net.nets)-i))) for i in range(len(Net.nets))]
curves[0].plot(-1,0)

runs=0

while not queue.empty():
    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    event = queue.get()
    t = event.time

    for i in range(len(curves)):
        curves[i].plot(pos=(t, Net.nets[i].graphValue()))

    [queue.put(ev) for ev in Net.nets[event.netID].update(event.value, t)]
    runs+=1
    rate(20)
    
for i in range(len(curves)):
    curves[i].plot(pos=(t, Net.nets[i].graphValue()))
for i in range(len(curves)):
    curves[i].plot(pos=(t+GATE_DELAY*2, Net.nets[i].graphValue()))


We can see that when the D net changes while E net is low, there is no effect on outputs (Q and QBar). But if D net changes while E net is high, then those changes are reflected in the outputs!

# Comments
I found that simulating combinational and sequential circuits were basically the same except for one issue I had with sequential circuits.

The only issue I had transitioning from combinational to sequential is that when you have undefined signals feeding back into the circuit they will then cause more undefined signals, which will lock the circuit into an undefined state. To get around this, I initialized the nets that feedback and cause the circuit to have memory (which in the D-Latch case is Q and QBar) to have values of 0 or 1 (in D-Latch case I initialized Q=0 and QBar=1)

Other than this issue my setup for combinational circuits worked immediately for sequential circuits with no other modifications!